In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer, make_column_selector
# !pip install category_encoders
from category_encoders import TargetEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

## Preprocessing

In [ ]:
# データロード
df = pd.read_csv('./workspace/vgsales.csv')

In [ ]:
df.info()

In [ ]:
# Yearの欠損数
len(df[df['Year'].isna()])

In [ ]:
# Publisherが欠損しているレコードのindex
pub_na_idx = df[df['Publisher'].isna()].index
# Yearが欠損しているレコードのindex
year_na_idx = df[df['Year'].isna()].index

In [ ]:
# Publisherの欠損を"NaN"で埋める
df[['Publisher']] = df[['Publisher']].fillna("NaN")

In [ ]:
# 複数のカラムを同時に埋めることも可能
df = pd.read_csv('./workspace/vgsales.csv')
df.fillna({'Publisher': "NaN", 'Year': df['Year'].median()}, inplace=True)
df.iloc[year_na_idx][:4]

In [ ]:
# Publisherの欠損とUnkownの分布の違いを確認
pub_nan_df = df[df['Publisher']=='NaN']
pub_unknown_df = df[df['Publisher']=='Unknown']
pub_missing_df = pd.concat([pub_nan_df, pub_unknown_df])
sns.pairplot(pub_missing_df, hue='Publisher')

In [ ]:
# SimpleImputerでも欠損値代入が可能
# Yearに中央値，Publisherには最頻値を入れる例
df = pd.read_csv('./workspace/vgsales.csv')
imputer = SimpleImputer(strategy='median')
df['Year'] = imputer.fit_transform(df[['Year']])
imputer = SimpleImputer(strategy='most_frequent')
# from IPython.display import display
# display(imputer.fit_transform(df[['Publisher']]))
# df['Publisher'] = imputer.fit_transform(df[['Publisher']])
df[['Publisher']] = imputer.fit_transform(df[['Publisher']])

In [ ]:
# Platform別にYearの中央値を計算し，その値で欠損値を埋める
df = pd.read_csv('./workspace/vgsales.csv')
platform_year_dict = df.groupby('Platform')['Year'].median().to_dict()
df['Year'] = df.apply(
    lambda row: platform_year_dict[row['Platform']] if row['Year'] in platform_year_dict and np.isnan(row['Year']) else row['Year'],
    axis=1
)

In [ ]:
# それぞれのPlatformに対応してYearの値が入っている
df.iloc[year_na_idx]

In [ ]:
# 代表値による欠損代入する場合，代表値は学習データを使用して計算する
df = pd.read_csv('./workspace/vgsales.csv')
df.drop('Global_Sales', inplace=True, axis=1) # Global_Salesがあると簡単にJP_Salesを計算できてしまうため
target = 'JP_Sales'
X = df.drop(target, axis=1)
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
platform_year_dict = X_train.groupby('Platform')['Year'].median().to_dict()
X_train['Year'] = X_train.apply(
    lambda row: platform_year_dict[row['Platform']] if np.isnan(row['Year']) and row['Platform'] in platform_year_dict else row['Year'],
    axis=1
)
# テストデータにも同様にplatform_year_dictを使用する
X_test['Year'] = X_test.apply(
    lambda row: platform_year_dict[row['Platform']] if np.isnan(row['Year']) and row['Platform'] in platform_year_dict else row['Year'],
    axis=1
)

In [ ]:
X_test

### kNNで欠損値代入

In [ ]:
# データ準備
df = pd.read_csv('./workspace/vgsales.csv')
# 欠損値をNaNで補完
df[['Publisher']] = 
df.drop("Name", inplace=True, axis=1)
# Yearカラムの欠損をKNNで代入する
target = "Year"
X = df.drop(target, axis=1)
y = df[target]
# 数値カラムのリスト取得(標準化の対象)
num_cols = 
# ダミー変数
X = 
# 標準化
"""
Xに本来の予測する対象データが含まれている場合,train_test_splitで分割し,
train_Xに対してStandardScalerを適用する
"""
X[num_cols] = 
# YearがNaNのデータはテストデータ，そうでなければ学習データ
test_indexes = 
train_indexes = 
X_train, X_test = 
y_train, y_test = 

In [ ]:
# kNNのモデルを作って予測値を代入する
knn = 
y_pred = 

In [ ]:
# 一つ目のテストデータのkNNのYear予測を確認
df.iloc[]

In [ ]:
y_pred[0]

In [ ]:
# neighborとして使用されたデータを確認する
neighbors = 
# neighbors[1][0]には，予測に使用されたデータのindexが格納されているが，このindexは学習データX_trainのindexであるため，
# 一度reset_index()でindexを振り直して対象データにアクセスし，['index']カラムから元のdfのindexを取得する
df.loc[]

In [ ]:
# kNNImputerを使う
imputer = 
imputer.
# ダミー変数
df = 
# 標準化
df[num_cols] = 
df_imputed = 

In [ ]:
df_imputed.iloc[test_indexes]

In [ ]:
# kNNImputerの結果とKNeighborsRegressorが等しいことを確認
y_pred

## 欠損値代入の比較

### データ準備とEDA

In [ ]:
df = pd.read_csv('./workspace/penguins_size.csv')
df.info()

In [ ]:
df.describe()

In [ ]:
%matplotlib inline
sns.pairplot(df, hue='species')

In [ ]:
sns.heatmap(df.corr(), annot=True)

In [ ]:
# カテゴリカルカラムのユニークな値とそれぞれの値にレコード数
cat_cols = 
for cat_col in cat_cols:
    print(f"======{cat_col}======")
    print(df[cat_col].value_counts())

In [ ]:
# "."は欠損値扱いにする
df.loc[] = np.nan
df[df['sex'].isna()]

In [ ]:
# それぞれの結果を格納するディクショナリー
results = {}

### 欠損値を落とすケース
- .dropna()
- モデルはなんでもOK(回答例ではロジスティック回帰を使用)
- 5foldx3で評価
- PipelineやColumnTransformerクラスを使用する
- 評価指標はloglossを使用

In [ ]:
df.dropna(inplace=True)
target = 'species'
X = df.drop(target, axis=1)
y = df[target]

#　前処理
# ダミー変数
X = 

# CV
k = 5
n_repeats = 3
cv = 

# Pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

pipeline = 
scores = 
results['drop'] = 

In [ ]:
results

### 欠損を新カテゴリーとする(カテゴリカル変数) and 中央値で代入(数値変数)
- 欠損値を新しいカテゴリとしたケース(数値カラムは中央値で代入)
- sklearn.impute.SimpleImputer() .fit_transform()

In [ ]:
df = pd.read_csv('penguins_size.csv')
# "."は欠損値扱いにする
df.loc[] = np.nan
target = 'species'
X = df.drop(target, axis=1)
y = df[target]

# CV
k = 5
n_repeats = 3
cv = 

# ダミー変数生成クラスを自作(Pipelineに組み込むため)
class GetDummies(BaseEstimator, TransformerMixin):


# PipelineはDataFrame全体に対しての処理
# 数値columnのみに適用することができない
# -> ColumnTransformer
# Columns Transformer (imputer)
num_cols = 
cat_cols = 
ct = 

# デフォルトだとColumnTransformerの結果がNumPyArrayになるが，
# 後続処理で問題になることがあるのでDataFrameにする
ct.

# Pipeline (dummy + scaler + model)
pipeline = 
scores = 
results['median'] = 
results

### 欠損をkNNで予測したケース(カテゴリカルカラムは最頻値)
- 欠損値をkNNで予測したケース(カテゴリカルカラムは最頻値で代入)
- sklearn.impute.KNNImputer(n_neighbors).fit_transform()

In [ ]:
df = pd.read_csv('penguins_size.csv')
# "."は欠損値扱いにする
df.loc[] = np.nan
target = 'species'
X = df.drop(target, axis=1)
y = df[target]

# CV
k = 5
n_repeats = 3
cv = 

# ダミー変数生成クラスを自作(Pipelineに組み込むため)
class GetDummies(BaseEstimator, TransformerMixin):

# Columns Transformer (imputer)
# num_cols = X.select_dtypes(include=np.number).columns.to_list()
# cat_cols = X.select_dtypes(exclude=np.number).columns.to_list()
num_pipeline = 
ct = 
ct.

# Pipeline (dummy + scaler + model)
pipeline = 
scores = 
results['knn'] = 
results

## Label Encoding

In [ ]:
df = pd.read_csv('penguins_size.csv')
oe = 
oe.
cat_cols = 
df[cat_cols] = 
df

## Target Encoding

In [ ]:
df = sns.load_dataset('titanic')
df.dropna(inplace=True)
# adult_maleのデータタイプをobjectに変更し，target encodingの対象とする(実際にはaloneも同様に行う
df['adult_male'] = 
df.info()

In [ ]:
encoder = 
encoder.
encoder.

In [ ]:
# マルチクラスのケース
df = pd.read_csv('penguins_size.csv')
df.loc[] = np.nan
targets = df['species'].unique()
for target in targets:

In [ ]:
df

## Target Encoding vs One Hot Encoding

### データ準備

In [ ]:
df = sns.load_dataset('titanic')
df.drop()

# adlut_maleとaloneをカテゴリカル変数として扱うための処理を書く
df[['adult_male', 'alone']] = 

X = df.drop('survived', axis=1)
y = df['survived']

In [ ]:
# それぞれの結果を格納
scores = {}

In [ ]:
# 欠損値代入->カテゴリカル変数のEncoding->標準化->モデル学習

# 処理する対象が違うので，カテゴリカルカラムと数値カラムのリストを取得する
cat_cols = 
num_cols = 

# 欠損値代入
cat_imputer = 
num_imputer = 
ct = 
ct.

# target encoding
pipeline_te = 

# one hot encoding
class GetDummies(BaseEstimator, TransformerMixin):
   
pipeline_ohe = 

cv = 
scores['target'] = 
scores['onehot'] = 

In [ ]:
scores

In [ ]:
# 結果をboxplotで描画
sns.boxplot()
plt.xticks()
plt.show()

In [ ]:
# 中央値比較
print(np.median(scores['target']))
print(np.median(scores['onehot']))

In [ ]:
# 平均値比較
print(np.mean(scores['target']))
print(np.mean(scores['onehot']))